# Sudoku - Resolução usando Constraint Satisfaction Problem (CSP)

## Membros da Equipe do Projeto:
- Ângelo de Carvalho Nunes
- Hyuri Fragoso Grande Silva
- Matheus Militão da Silva Sulzbacher


## a) Modelagem

### Problema de Satisfação de Restrições (CSP)

O Sudoku foi modelado como um **Constraint Satisfaction Problem (CSP)** com os seguintes componentes:

#### **Variáveis (X):**
- Cada célula vazia do tabuleiro 9×9 representa uma variável
- Total de até 81 variáveis (dependendo do puzzle inicial)

#### **Domínio (D):**
- Para cada variável: D = {1, 2, 3, 4, 5, 6, 7, 8, 9}
- Domínio reduzido dinamicamente conforme restrições

#### **Restrições (C):**
1. **Restrição de Linha:** Cada linha deve conter todos os números de 1 a 9 (AllDifferent)
2. **Restrição de Coluna:** Cada coluna deve conter todos os números de 1 a 9 (AllDifferent)
3. **Restrição de Quadrado 3×3:** Cada um dos 9 quadrados deve conter todos os números de 1 a 9 (AllDifferent)

### Algoritmo de Resolução

**Backtracking com Heurísticas:**
- **MRV (Minimum Remaining Values):** Seleciona a variável com menor número de valores possíveis
- **Forward Checking:** Verifica consistência das restrições antes da atribuição
- **Backtracking:** Desfaz atribuições inconsistentes e explora alternativas


## b) Implementação


In [141]:
import numpy as np
import time

class SudokuSolver:
    def __init__(self):
        self.board = np.zeros((9,9), np.int64)
        self.backtrack_count = 0
        self.recursive_calls = 0
        
    def set(self, row, col, value):
        """Define um valor em uma posição específica do tabuleiro"""
        if 0 <= row < 9 and 0 <= col < 9:
            self.board[row, col] = value
            return True
        return False
    
    def set_all(self, values):
        """Define múltiplos valores no tabuleiro"""
        for value in values:
            self.set(value[0], value[1], value[2])
    
    def is_valid_move(self, row, col, num):
        """
        Verifica se é válido colocar um número numa posição específica
        Implementa as três restrições do Sudoku
        """
        # Restrição 1: Verificar linha
        for c in range(9):
            if self.board[row, c] == num:
                return False
        
        # Restrição 2: Verificar coluna
        for r in range(9):
            if self.board[r, col] == num:
                return False
        
        # Restrição 3: Verificar quadrado 3x3
        start_row = (row // 3) * 3
        start_col = (col // 3) * 3
        for r in range(start_row, start_row + 3):
            for c in range(start_col, start_col + 3):
                if self.board[r, c] == num:
                    return False
        
        return True
    
    def find_empty_cell_mrv(self):
        """
        Heurística MRV (Minimum Remaining Values)
        Encontra a célula vazia com menor número de valores possíveis
        """
        min_options = 10
        best_cell = None
        
        for row in range(9):
            for col in range(9):
                if self.board[row, col] == 0:
                    # Conta quantos números são possíveis nesta célula
                    options = 0
                    for num in range(1, 10):
                        if self.is_valid_move(row, col, num):
                            options += 1
                    
                    # Se encontrou célula com menos opções, atualiza
                    if options < min_options:
                        min_options = options
                        best_cell = (row, col)
                        
                        # Se só há uma opção, retorna imediatamente
                        if options == 1:
                            return best_cell
        
        return best_cell
    
    def get_possible_values(self, row, col):
        """
        Forward Checking: Retorna lista de valores possíveis para uma célula
        """
        if self.board[row, col] != 0:
            return []
        
        possible = []
        for num in range(1, 10):
            if self.is_valid_move(row, col, num):
                possible.append(num)
        
        return possible
    
    def solve_backtracking(self):
        """
        Algoritmo principal: Backtracking com heurística MRV
        Conta chamadas recursivas e backtracks para análise de desempenho
        """
        self.recursive_calls += 1
        
        # Encontra próxima célula vazia usando heurística MRV
        empty_cell = self.find_empty_cell_mrv()
        
        # Caso base: Se não há células vazias, puzzle está resolvido
        if empty_cell is None:
            return True
        
        row, col = empty_cell
        
        # Obtém valores possíveis (Forward Checking)
        possible_values = self.get_possible_values(row, col)
        
        # Se não há valores possíveis, força backtrack
        if not possible_values:
            self.backtrack_count += 1
            return False
        
        # Tenta cada valor possível
        for num in possible_values:
            # Faz a atribuição
            self.board[row, col] = num
            
            # Recursivamente tenta resolver o resto
            if self.solve_backtracking():
                return True
            
            # Backtrack: desfaz a jogada
            self.board[row, col] = 0
            self.backtrack_count += 1
        
        # Nenhum valor funcionou
        return False
    
    def solve(self):
        """
        Método público para resolver o Sudoku
        Reinicia contadores e preserva estado original em caso de falha
        """
        # Reinicia contadores de desempenho
        self.backtrack_count = 0
        self.recursive_calls = 0
        
        # Salva estado original
        original_board = self.board.copy()
        
        # Mede tempo de execução
        start_time = time.time()
        
        # Tenta resolver usando backtracking
        success = self.solve_backtracking()
        
        end_time = time.time()
        execution_time = end_time - start_time
        
        if success:
            return True, execution_time
        else:
            # Restaura estado original em caso de falha
            self.board = original_board
            return False, execution_time
    
    def get_performance_stats(self):
        """
        Retorna estatísticas de desempenho da última execução
        """
        return {
            'recursive_calls': self.recursive_calls,
            'backtrack_count': self.backtrack_count
        }
    
    def is_solution_valid(self):
        """
        Verifica se a solução atual é válida
        """
        # Verificar linhas
        for row in range(9):
            if len(set(self.board[row,:])) != 9 or 0 in self.board[row,:]:
                return False, f"Linha {row} inválida"
        
        # Verificar colunas  
        for col in range(9):
            if len(set(self.board[:,col])) != 9 or 0 in self.board[:,col]:
                return False, f"Coluna {col} inválida"
        
        # Verificar quadrados 3x3
        for box_row in range(3):
            for box_col in range(3):
                start_row = box_row * 3
                start_col = box_col * 3
                box = self.board[start_row:start_row+3, start_col:start_col+3].flatten()
                if len(set(box)) != 9 or 0 in box:
                    return False, f"Quadrado ({box_row},{box_col}) inválido"
        
        return True, "Solução válida"
    
    def __str__(self):
        """
        Representação visual do tabuleiro
        """
        rows = []
        rows.append('#+------+-------+------+')
        
        for y in range(9):
            row = []
            for e in self.board[y,:]:
                if e == 0:
                    row.append(' ')
                else:
                    row.append(str(e))
            
            # Adiciona separadores verticais
            row.insert(3, '|')
            row.insert(7, '|')
            rows.append('#|' + ' '.join(row) + '|')
            
            # Adiciona separadores horizontais
            if y == 2 or y == 5 or y == 8:
                rows.append('#+------+-------+------+')
        
        return '\n'.join(rows)

# Classe compatível com a interface fornecida pelo professor
class SudokuBoard(SudokuSolver):
    def __init__(self):
        super().__init__()


## c) Apresentação de Soluções - Quatro Jogos Distintos

### Função auxiliar para apresentar resultados

In [142]:
def solve_and_display(board, puzzle_name, initial_values):
    """
    Função auxiliar para resolver e exibir resultados de um puzzle
    """
    print(f"\n{'='*60}")
    print(f" {puzzle_name}")
    print(f"{'='*60}")
    
    # Configura o puzzle inicial
    board.set_all(initial_values)
    
    print("\n PUZZLE INICIAL:")
    print(board)
    
    success, execution_time = board.solve()
    
    if success:
        print("\n SOLUÇÃO ENCONTRADA:")
        print(board)
        
        # Verifica validade da solução
        is_valid, validation_msg = board.is_solution_valid()
        print(f"\n VALIDAÇÃO: {validation_msg}")
        
        # Exibe estatísticas de desempenho
        stats = board.get_performance_stats()
        print("\n DESEMPENHO:")
        print(f"     Tempo de execução: {execution_time:.4f} segundos")
        print(f"     Chamadas recursivas: {stats['recursive_calls']}")
        print(f"     Backtracks realizados: {stats['backtrack_count']}")
        
    else:
        print("\n NÃO É POSSÍVEL RESOLVER ESTE PUZZLE")
        print(f"     Tempo tentado: {execution_time:.4f} segundos")
        stats = board.get_performance_stats()
        print(f"     Chamadas recursivas: {stats['recursive_calls']}")
        print(f"     Backtracks realizados: {stats['backtrack_count']}")


### Jogo 1 - Nível Fácil


In [143]:
# JOGO 1 - NÍVEL FÁCIL
board1 = SudokuBoard()

puzzle1_values = [
    [0,0,5], [0,1,3], [0,4,7],
    [1,0,6], [1,3,1], [1,4,9], [1,5,5],
    [2,1,9], [2,2,8], [2,7,6],
    [3,0,8], [3,4,6], [3,8,3],
    [4,0,4], [4,3,8], [4,5,3], [4,8,1],
    [5,0,7], [5,4,2], [5,8,6],
    [6,1,6], [6,6,2], [6,7,8],
    [7,3,4], [7,4,1], [7,5,9], [7,8,5],
    [8,4,8], [8,7,7], [8,8,9]
]

solve_and_display(board1, "JOGO 1 - NÍVEL FÁCIL", puzzle1_values)



 JOGO 1 - NÍVEL FÁCIL

 PUZZLE INICIAL:
#+------+-------+------+
#|5 3   |   7   |      |
#|6     | 1 9 5 |      |
#|  9 8 |       |   6  |
#+------+-------+------+
#|8     |   6   |     3|
#|4     | 8   3 |     1|
#|7     |   2   |     6|
#+------+-------+------+
#|  6   |       | 2 8  |
#|      | 4 1 9 |     5|
#|      |   8   |   7 9|
#+------+-------+------+

 SOLUÇÃO ENCONTRADA:
#+------+-------+------+
#|5 3 4 | 6 7 8 | 9 1 2|
#|6 7 2 | 1 9 5 | 3 4 8|
#|1 9 8 | 3 4 2 | 5 6 7|
#+------+-------+------+
#|8 5 9 | 7 6 1 | 4 2 3|
#|4 2 6 | 8 5 3 | 7 9 1|
#|7 1 3 | 9 2 4 | 8 5 6|
#+------+-------+------+
#|9 6 1 | 5 3 7 | 2 8 4|
#|2 8 7 | 4 1 9 | 6 3 5|
#|3 4 5 | 2 8 6 | 1 7 9|
#+------+-------+------+

 VALIDAÇÃO: Solução válida

 DESEMPENHO:
     Tempo de execução: 0.0058 segundos
     Chamadas recursivas: 52
     Backtracks realizados: 0


### Jogo 2 - Nível Médio


In [144]:
# JOGO 2 - NÍVEL MÉDIO
board2 = SudokuBoard()

puzzle2_values = [
    [0,1,9], [0,4,5],
    [1,0,8], [1,7,6],
    [2,8,3],
    [3,0,1], [3,4,6], [3,8,5],
    [4,0,5], [4,1,8], [4,7,1], [4,8,9],
    [5,0,2], [5,4,4], [5,8,7],
    [6,0,6],
    [7,1,2], [7,8,1],
    [8,4,8], [8,7,4]
]

solve_and_display(board2, "JOGO 2 - NÍVEL MÉDIO", puzzle2_values)



 JOGO 2 - NÍVEL MÉDIO

 PUZZLE INICIAL:
#+------+-------+------+
#|  9   |   5   |      |
#|8     |       |   6  |
#|      |       |     3|
#+------+-------+------+
#|1     |   6   |     5|
#|5 8   |       |   1 9|
#|2     |   4   |     7|
#+------+-------+------+
#|6     |       |      |
#|  2   |       |     1|
#|      |   8   |   4  |
#+------+-------+------+

 SOLUÇÃO ENCONTRADA:
#+------+-------+------+
#|3 9 2 | 6 5 8 | 1 7 4|
#|8 1 7 | 9 3 4 | 5 6 2|
#|4 5 6 | 2 1 7 | 9 8 3|
#+------+-------+------+
#|1 7 3 | 8 6 9 | 4 2 5|
#|5 8 4 | 7 2 3 | 6 1 9|
#|2 6 9 | 1 4 5 | 8 3 7|
#+------+-------+------+
#|6 4 5 | 3 7 1 | 2 9 8|
#|7 2 8 | 4 9 6 | 3 5 1|
#|9 3 1 | 5 8 2 | 7 4 6|
#+------+-------+------+

 VALIDAÇÃO: Solução válida

 DESEMPENHO:
     Tempo de execução: 0.0460 segundos
     Chamadas recursivas: 212
     Backtracks realizados: 164


### Jogo 3 - Nível Difícil


In [145]:
# JOGO 3 - NÍVEL DIFÍCIL  
board3 = SudokuBoard()

puzzle3_values = [
    [0,4,2],
    [1,0,8], [1,3,4], [1,6,1],
    [2,1,4], [2,2,1], [2,5,8],
    [3,0,6], [3,7,4], [3,8,7],
    [4,2,8], [4,6,6],
    [5,0,1], [5,1,9], [5,8,2],
    [6,3,1], [6,6,4], [6,7,6],
    [7,2,6], [7,5,4], [7,8,1],
    [8,4,5]
]

solve_and_display(board3, "JOGO 3 - NÍVEL DIFÍCIL", puzzle3_values)



 JOGO 3 - NÍVEL DIFÍCIL

 PUZZLE INICIAL:
#+------+-------+------+
#|      |   2   |      |
#|8     | 4     | 1    |
#|  4 1 |     8 |      |
#+------+-------+------+
#|6     |       |   4 7|
#|    8 |       | 6    |
#|1 9   |       |     2|
#+------+-------+------+
#|      | 1     | 4 6  |
#|    6 |     4 |     1|
#|      |   5   |      |
#+------+-------+------+

 SOLUÇÃO ENCONTRADA:
#+------+-------+------+
#|9 6 5 | 7 2 1 | 8 3 4|
#|8 7 2 | 4 6 3 | 1 5 9|
#|3 4 1 | 5 9 8 | 2 7 6|
#+------+-------+------+
#|6 2 3 | 8 1 5 | 9 4 7|
#|4 5 8 | 2 7 9 | 6 1 3|
#|1 9 7 | 3 4 6 | 5 8 2|
#+------+-------+------+
#|2 3 9 | 1 8 7 | 4 6 5|
#|5 8 6 | 9 3 4 | 7 2 1|
#|7 1 4 | 6 5 2 | 3 9 8|
#+------+-------+------+

 VALIDAÇÃO: Solução válida

 DESEMPENHO:
     Tempo de execução: 0.0254 segundos
     Chamadas recursivas: 100
     Backtracks realizados: 44


### Jogo 4 - Puzzle Mínimo (17 pistas)


In [146]:
# JOGO 4 - PUZZLE MÍNIMO (apenas 17 pistas - mínimo matemático para Sudoku único)
board4 = SudokuBoard()

puzzle4_values = [
    [0,0,1],
    [1,2,2], [1,4,3],
    [2,1,4], [2,6,5],
    [3,3,6], [3,7,7],
    [4,0,8], [4,8,9],
    [5,1,1], [5,5,2],
    [6,2,3], [6,7,4],
    [7,4,5], [7,6,6],
    [8,8,7]
]

solve_and_display(board4, "JOGO 4 - PUZZLE MÍNIMO (17 PISTAS)", puzzle4_values)



 JOGO 4 - PUZZLE MÍNIMO (17 PISTAS)

 PUZZLE INICIAL:
#+------+-------+------+
#|1     |       |      |
#|    2 |   3   |      |
#|  4   |       | 5    |
#+------+-------+------+
#|      | 6     |   7  |
#|8     |       |     9|
#|  1   |     2 |      |
#+------+-------+------+
#|    3 |       |   4  |
#|      |   5   | 6    |
#|      |       |     7|
#+------+-------+------+

 SOLUÇÃO ENCONTRADA:
#+------+-------+------+
#|1 3 9 | 7 6 5 | 4 2 8|
#|5 8 2 | 4 3 9 | 7 1 6|
#|6 4 7 | 2 8 1 | 5 9 3|
#+------+-------+------+
#|3 9 4 | 6 1 8 | 2 7 5|
#|8 2 5 | 3 4 7 | 1 6 9|
#|7 1 6 | 5 9 2 | 3 8 4|
#+------+-------+------+
#|2 5 3 | 9 7 6 | 8 4 1|
#|9 7 1 | 8 5 4 | 6 3 2|
#|4 6 8 | 1 2 3 | 9 5 7|
#+------+-------+------+

 VALIDAÇÃO: Solução válida

 DESEMPENHO:
     Tempo de execução: 0.0277 segundos
     Chamadas recursivas: 81
     Backtracks realizados: 17


## d) Análise de Desempenho

### Resumo Comparativo dos Quatro Jogos


In [147]:
print("\n" + "="*65)
print("ANÁLISE COMPARATIVA DE DESEMPENHO")
print("="*65)

# Dados dos jogos (serão preenchidos após execução)
games_data = [
    {"name": "Jogo 1 - Fácil", "board": board1},
    {"name": "Jogo 2 - Médio", "board": board2}, 
    {"name": "Jogo 3 - Difícil", "board": board3},
    {"name": "Jogo 4 - Mínimo", "board": board4}
]

print(f"{'Jogo':<20} {'Rec. Calls':<12} {'Backtracks':<12} {'Taxa Backtrack':<15}")
print("-" * 65)

for game in games_data:
    stats = game["board"].get_performance_stats()
    recursive_calls = stats['recursive_calls']
    backtracks = stats['backtrack_count']
    
    # Calcula taxa de backtrack (menor é melhor)
    if recursive_calls > 0:
        backtrack_rate = f"{(backtracks/recursive_calls)*100:.1f}%"
    else:
        backtrack_rate = "N/A"
    
    print(f"{game['name']:<20} {recursive_calls:<12} {backtracks:<12} {backtrack_rate:<15}")

print("\n INTERPRETAÇÃO DOS RESULTADOS:")
print("- Chamadas Recursivas: Total de nós explorados na árvore de busca")
print("- Backtracks: Número de retrocessos necessários (falhas)")
print("- Taxa de Backtrack: Percentual de backtracks em relação às chamadas (menor = melhor)")

print("\n EFICÁCIA DAS HEURÍSTICAS:")
print("- MRV reduz significativamente o fator de ramificação")
print("- Forward Checking detecta inconsistências precocemente")
print("- Puzzles com mais restrições iniciais são resolvidos mais rapidamente")



ANÁLISE COMPARATIVA DE DESEMPENHO
Jogo                 Rec. Calls   Backtracks   Taxa Backtrack 
-----------------------------------------------------------------
Jogo 1 - Fácil       52           0            0.0%           
Jogo 2 - Médio       212          164          77.4%          
Jogo 3 - Difícil     100          44           44.0%          
Jogo 4 - Mínimo      81           17           21.0%          

 INTERPRETAÇÃO DOS RESULTADOS:
- Chamadas Recursivas: Total de nós explorados na árvore de busca
- Backtracks: Número de retrocessos necessários (falhas)
- Taxa de Backtrack: Percentual de backtracks em relação às chamadas (menor = melhor)

 EFICÁCIA DAS HEURÍSTICAS:
- MRV reduz significativamente o fator de ramificação
- Forward Checking detecta inconsistências precocemente
- Puzzles com mais restrições iniciais são resolvidos mais rapidamente


## e) Célula Teste Final


In [148]:
# CÉLULA TESTE FINAL - Para avaliação do Professor

# Exemplo de puzzle para teste 
final_board = SudokuBoard()

# Puzzle de teste padrão (modificar esses valores para testar outros puzzles)
test_values = [
    [0,0,5], [0,1,3], [0,4,7],
    [1,0,6], [1,3,1], [1,4,9], [1,5,5],
    [2,1,9], [2,2,8], [2,7,6],
    [3,0,8], [3,4,6], [3,8,3],
    [4,0,4], [4,3,8], [4,5,3], [4,8,1],
    [5,0,7], [5,4,2], [5,8,6],
    [6,1,6], [6,6,2], [6,7,8],
    [7,3,4], [7,4,1], [7,5,9], [7,8,5],
    [8,4,8], [8,7,7], [8,8,9]
]

print("TESTE FINAL")
print("="*50)

# Configurar o puzzle
final_board.set_all(test_values)

print("\nPUZZLE INICIAL:")
print(final_board)

start_time = time.time()
success, execution_time = final_board.solve()
end_time = time.time()

if success:
    print("\nSOLUÇÃO FINAL:")
    print(final_board)
    
    # Validar solução
    is_valid, validation_msg = final_board.is_solution_valid()
    print(f"\nVALIDAÇÃO: {validation_msg}")
    
    # Exibir desempenho detalhado
    stats = final_board.get_performance_stats()
    print("\nDESEMPENHO:")
    print(f"    Tempo de execução: {execution_time:.6f} segundos")
    print(f"    Chamadas recursivas: {stats['recursive_calls']}")
    print(f"    Backtracks realizados: {stats['backtrack_count']}")
    
    if stats['recursive_calls'] > 0:
        backtrack_rate = (stats['backtrack_count'] / stats['recursive_calls']) * 100
        print(f"    Taxa de backtrack: {backtrack_rate:.2f}%")
    
    # Análise da complexidade
    empty_cells = np.count_nonzero(final_board.board == 0)
    if empty_cells == 0:  # Puzzle resolvido
        initial_empty = 81 - len(test_values)
        print(f"    Células inicialmente vazias: {initial_empty}")
        
else:
    print("\nFALHA NA RESOLUÇÃO")
    print(f"    Tempo tentado: {execution_time:.6f} segundos")
    stats = final_board.get_performance_stats()
    print(f"    Chamadas recursivas: {stats['recursive_calls']}")
    print(f"    Backtracks realizados: {stats['backtrack_count']}")
    print("    Possível puzzle inválido ou sem solução")


TESTE FINAL

PUZZLE INICIAL:
#+------+-------+------+
#|5 3   |   7   |      |
#|6     | 1 9 5 |      |
#|  9 8 |       |   6  |
#+------+-------+------+
#|8     |   6   |     3|
#|4     | 8   3 |     1|
#|7     |   2   |     6|
#+------+-------+------+
#|  6   |       | 2 8  |
#|      | 4 1 9 |     5|
#|      |   8   |   7 9|
#+------+-------+------+

SOLUÇÃO FINAL:
#+------+-------+------+
#|5 3 4 | 6 7 8 | 9 1 2|
#|6 7 2 | 1 9 5 | 3 4 8|
#|1 9 8 | 3 4 2 | 5 6 7|
#+------+-------+------+
#|8 5 9 | 7 6 1 | 4 2 3|
#|4 2 6 | 8 5 3 | 7 9 1|
#|7 1 3 | 9 2 4 | 8 5 6|
#+------+-------+------+
#|9 6 1 | 5 3 7 | 2 8 4|
#|2 8 7 | 4 1 9 | 6 3 5|
#|3 4 5 | 2 8 6 | 1 7 9|
#+------+-------+------+

VALIDAÇÃO: Solução válida

DESEMPENHO:
    Tempo de execução: 0.006025 segundos
    Chamadas recursivas: 52
    Backtracks realizados: 0
    Taxa de backtrack: 0.00%
    Células inicialmente vazias: 51


## Conclusão

### Resumo da Implementação

Esta implementação demonstra a aplicação eficaz de **Constraint Satisfaction Problem (CSP)** para resolver puzzles Sudoku. Os principais pontos são:

#### **Técnicas Utilizadas:**
1. **Backtracking:** Algoritmo de busca sistemática com retrocesso
2. **MRV (Minimum Remaining Values):** Heurística de seleção de variável
3. **Forward Checking:** Verificação antecipada de consistência
4. **Constraint Propagation:** Redução do domínio baseada em restrições

#### **Vantagens da Abordagem:**
- **Completude:** Garante encontrar solução se existir
- **Otimalidade:** Encontra soluções válidas respeitando todas as restrições
- **Eficiência:** Heurísticas reduzem significativamente o espaço de busca
- **Robustez:** Detecta puzzles impossíveis ou mal formulados

#### **Métricas de Desempenho:**
- **Chamadas Recursivas:** Mede a exploração do espaço de estados
- **Backtracks:** Quantifica retrocessos necessários
- **Tempo de Execução:** Performance temporal do algoritmo

A implementação demonstra que CSP é uma abordagem natural e eficiente para problemas de satisfação de restrições como o Sudoku, combinando fundamentos teóricos sólidos com otimizações práticas.
